In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [10]:
from sqlalchemy import create_engine

# MySQL 데이터베이스 연결 정보
user = 'root'
password = '1234'
host = 'localhost'
database = 'trouver'

# SQLAlchemy 엔진 생성
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

In [2]:
import tensorflow as tf
path = './dataset'

# Dataset
* rating.csv
* travel.csv
* users : 자체 DB 사용 예정 (MySQL)
> 출처 : BC카드 제주 관광객 소비패턴 변화 분석, 
> visitJEJU 기반 데이터 수집

In [3]:
os.listdir(path)

['rating.csv', 'travel.csv', 'visitjeju_data.xlsx']

In [5]:
#여행지 데이터프레임
def load_ratings():
    col_name = ['placeid','title','tag']
    travel_df = pd.read_csv(os.path.join(path,"ratings.csv"),sep=',', header=None, engine='python', names=col_name)
    travel_df.head()

,placeid,contentsid,contentscd,title,regioncd_value,regioncd_label,region1cd_value,region1cd_label,region2cd_value,region2cd_label,tag
0,1,CNTS_200000000015109,c2,숲을걷다,1,북부,region1,제주시,12,애월,"애월, 소품샵, 감귤모자, 의류, 엽서, 잡화"
1,2,CNTS_000000000021301,c2,아코하루,1,북부,region1,제주시,11,제주시내,"쇼핑,공방,기념품,공용주차장,현금결제,카드결제,무료 WIFI,편의점"
2,3,CNTS_200000000007257,c2,풍년상회,1,북부,region1,제주시,15,조천,"쇼핑,기념품"
3,4,CNTS_200000000013168,c5,세계자동차&피아노박물관 4th 봄날의 향연 뮤지엄콘서트,4,남부,region2,서귀포시,23,안덕,"축제, 문화관광"
4,6,CNTS_200000000014956,c4,R14,3,서부,region1,제주시,13,한림,"한림, 월령리, 스콘, 백년초, 아이스크림, 콜드브루, 허브차, 생딸기우유,바닐라빈 라떼"


In [8]:
#유저 평가 데이터프레임
rating_df = pd.read_csv(path+'/rating.csv')
rating_df.head()

,age_id,regioncd_value,pla_r_rate,pla_r_date
0,1,남부,5.8,1차년도
1,1,남부,7.4,2차년도
2,1,동부,6.6,1차년도
3,1,동부,8.5,2차년도
4,1,북부,7.6,1차년도


임의로 사용자의 나이대에 따라 id 부여함.  
[참고 Notion Link 🔗](https://www.notion.so/jxxngxnt/44e0fcdeb8814a0ea9e12de162137637?pvs=4#cafa46a992c94de2a44dd8b375933ac6)

In [18]:
#유저 데이터프레임
query = '''
SELECT 
    BIN_TO_UUID(user_id) as user_id ,user_name, user_phone,  
    user_birth, user_mbti, 
    user_pay, user_login_id, user_login_pw, 
    @age := FLOOR(DATEDIFF(NOW(), user_birth) / 365.25) AS age,
    CONCAT(
        CASE
            WHEN user_gender=0 THEN '남성'
            WHEN user_gender=1 THEN '여성'
            ELSE 'etc.'
        END,
        ' ',
        CASE
            WHEN @age <=29 THEN '20대 이하'
            WHEN @age BETWEEN 30 AND 39 THEN '30대'
            WHEN @age BETWEEN 40 AND 49 THEN '40대'
            WHEN @age BETWEEN 50 AND 59 THEN '50대'
            WHEN @age >=60 THEN '60대 이상'
            ELSE 'etc.'
        END
    ) AS user_age_group
FROM user'''  # 실행할 SQL 쿼리
users = pd.read_sql(query, engine)
print(users.head())

                                user_id user_name     user_phone  user_birth  \
0  d3ca4a15-2a0c-11ef-9ffb-7085c2d2eea0      test  010-0000-0000  2000-01-01   

  user_mbti user_pay user_login_id user_login_pw  age user_age_group  
0      ENFP  결제정보 없음          test          test   24      남성 20대 이하  
